You can check all the files in this [link](https://console.cloud.google.com/storage/browser/t5_convert_tranformers?hl=de&project=indigo-gecko-270520&pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&prefix=&forceOnObjectsSortingFiltering=false).




In [ ]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5==0.6.4

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

BASE_DIR = "gs://t5_convert_tranformers " #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "v2-8"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  tf.config.experimental_connect_to_host(TPU_ADDRESS)

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
     |████████████████████████████████| 163kB 943kB/s 
     |████████████████████████████████| 2.6MB 3.1MB/s 
     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 348kB 13.5MB/s 
     |████████████████████████████████| 1.1MB 16.0MB/s 
     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 3.6MB 45.7MB/s 
     |████████████████████████████████| 890kB 44.7MB/s 
     |████████████████████████████████| 3.0MB 46.8MB/s 
INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
Setting up GCS access...
Running on TPU: grpc://10.96.34.58:8470
Instructions for updating:
non-resource variables are not supported in the long term


# Part I: Using decode mode of mesh_transformer_main to load the checkpoint and predict


### Give input example

In [ ]:
code = "function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }"

codes = [code]

inputs_path = 'input.txt'
with tf.io.gfile.GFile(inputs_path, "w") as f:
  for c in codes:
    f.write("function documentation generation javascript: %s\n" % c)

In [ ]:
!wget https://storage.googleapis.com/t5_convert_tranformers/t5_code_tasks_colab_public.py

--2020-11-12 15:52:18--  https://storage.googleapis.com/t5_convert_tranformers/t5_code_tasks_colab_public.py
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11196 (11K) [text/x-python-script]
Saving to: ‘t5_code_tasks_colab_public.py’

t5_code_tasks_colab 100%[===================>]  10.93K  --.-KB/s    in 0s      

2020-11-12 15:52:19 (43.7 MB/s) - ‘t5_code_tasks_colab_public.py’ saved [11196/11196]



### Decode and give the prediction

In [ ]:
!python -m t5.models.mesh_transformer_main \
  --module_import="t5_code_tasks_colab_public" \
  --tpu=$TPU_ADDRESS \
  --model_dir="gs://t5_convert_tranformers/model" \
  --gin_file="gs://t5_convert_tranformers/model/operative_config.gin" \
  --gin_file="infer.gin" \
  --gin_file="beam_search.gin" \
  --gin_param="input_filename = 'input.txt'"\
  --gin_param="output_filename = 'mesh_tranformer_output.txt'"\
  --gin_param="utils.tpu_mesh_shape.tpu_topology = 'v2-8'"\
  --gin_param="infer_checkpoint_step = 16000"

2020-10-15 11:08:05.332290: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term
2020-10-15 11:08:08.730369: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2020-10-15 11:08:08.882700: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
Vocab has a size of 32100

2020-10-15 11:08:09.458105: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2020-10-15 11:08:09.588867: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was 

### Read output

In [ ]:
prediction_file = 'mesh_tranformer_output.txt-16000'

print("\nPredictions using checkpoint 16000:\n" )
with tf.io.gfile.GFile(prediction_file) as f:
  for q, a in zip(codes, f):
    if q:
      print("Code: " + q)
      print("Document for code: " + a)
      print()

print("Gold: determine if we re running in a standard browser environment")


Predictions using checkpoint 16000:

Code: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }
Document for code: Returns true if the browser is a native element .


Gold: determine if we re running in a standard browser environment


# Part II: Using MtfModel to load the checkpoint and predict

In [ ]:
from t5.data.sentencepiece_vocabulary import SentencePieceVocabulary

vocab_model_path = 'gs://t5_convert_tranformers/spm/code_spm_unigram_40M.model'
vocab = SentencePieceVocabulary(vocab_model_path, extra_ids=100)

print("Vocab has a size of %d\n" % vocab.vocab_size)

Vocab has a size of 32100



### Set the task

In [ ]:
def javascript_codeSearchNet_dataset_fn(split, shuffle_files=False):
    del shuffle_files

    ds = tf.data.TextLineDataset(javascript_path[split])
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""], field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    ds = ds.map(lambda *ex: dict(zip(["code", "docstring"], ex)))
    return ds


def javascript_preprocessor(ds):
    def normalize_text(text):
        # text = tf.strings.lower(text)
        return text

    def to_inputs_and_targets(ex):
        return {
            "inputs": tf.strings.join(["function documentation generation javascript: ", normalize_text(ex["code"])]),
            "targets": normalize_text(ex["docstring"])
        }

    return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)


t5.data.TaskRegistry.remove('function_documentation_generation_javascript_code')
t5.data.TaskRegistry.add(
    "function_documentation_generation_javascript_code",
    dataset_fn=javascript_codeSearchNet_dataset_fn,
    output_features={
        "inputs": t5.data.utils.Feature(vocabulary=vocab),
        "targets": t5.data.utils.Feature(vocabulary=vocab),
    },
    splits=["train", "validation"],
    text_preprocessor=[javascript_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.bleu, t5.evaluation.metrics.accuracy, t5.evaluation.metrics.rouge],
    # num_input_examples=num_review_examples
)

In [ ]:
t5.data.MixtureRegistry.remove("function_documentation_generation")
t5.data.MixtureRegistry.add(
    "function_documentation_generation",
    [('function_documentation_generation_javascript_code', 58025.0)]
)

### Set Model

In [ ]:
MODEL_SIZE = "small" #@param["small", "base", "large", "3B", "11B"]
# Public GCS path for T5 pre-trained model checkpoints
MODEL_DIR = "gs://t5_convert_tranformers/model"


# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 128, "targets": 32},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

### Give model code example and get prediction

In [ ]:
predict_outputs_path = 'MtfModel-output.txt'

with tf_verbosity_level('ERROR'):
  model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
  model.predict(
      input_file="input.txt",
      output_file=predict_outputs_path,
      checkpoint_steps=16000,
      beam_size=4,
      vocabulary=vocab, 
      # Select the most probable output token at each step.
      temperature=0,
  )


In [ ]:
prediction_file = "MtfModel-output.txt-16000"
print("\nPredictions using checkpoint 16000:\n" )
with tf.io.gfile.GFile(prediction_file) as f:
  for q, a in zip(codes, f):
    if q:
      print("Code: " + q)
      print("Documentation: " + a)
      print()

print("Gold: determine if we re running in a standard browser environment")


Predictions using checkpoint 16000:

Code: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }
Documentation: Returns true if the browser is a native element .


Gold: determine if we re running in a standard browser environment


# Part III: Convert the checkpoint to transformers Pytorch and check the prediction output

### install the special branch

In [ ]:
!pip install git+https://github.com/agemagician/transformers.git@adapt_t5_for_covid_19_3b
#!pip install git+https://github.com/agemagician/transformers.git@patch-3

#!pip install git+https://github.com/patrickvonplaten/transformers.git@t5_v1_1

  Cloning https://github.com/patrickvonplaten/transformers.git (to revision t5_v1_1) to /tmp/pip-req-build-ak3ux8m1
  Running command git clone -q https://github.com/patrickvonplaten/transformers.git /tmp/pip-req-build-ak3ux8m1
  Running command git checkout -b t5_v1_1 --track origin/t5_v1_1
  Switched to a new branch 't5_v1_1'
  Branch 't5_v1_1' set up to track remote branch 't5_v1_1' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 5.3MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 
     |████████████████████████████████| 890kB 40.4MB/s 
  Created wheel for transformers: filename=transformers-3.5.0-cp36-none-any.whl size=1317504 sha256=c7fdb077c80d400cd7a3815dda5159c56ec15370ad0209f2d64f1b9a71ae4c33
  Stored in directory: /tmp/pip-ephem-wheel-cache-0tvxfgvf/wheels/59/0e/d1/4b7c8f0a1dba00d281802a0a067b6c66b20362769859faa109
Successfully

In [ ]:
!transformers-cli env

2020-10-15 09:29:02.825347: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2020-10-15 09:29:04.388166: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2249995000 Hz
2020-10-15 09:29:04.388443: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x657c680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-15 09:29:04.388469: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-10-15 09:29:04.390003: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-15 09:29:04.398487: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-15 09:29:04.398526: I tensorf

In [ ]:
!mkdir checkpoint

In [ ]:
!wget https://storage.googleapis.com/t5_convert_tranformers/model/checkpoint -P checkpoint/
!wget https://storage.googleapis.com/t5_convert_tranformers/model/graph.pbtxt -P checkpoint/
!wget https://storage.googleapis.com/t5_convert_tranformers/model/model.ckpt-16000.data-00000-of-00002 -P checkpoint/
!wget https://storage.googleapis.com/t5_convert_tranformers/model/model.ckpt-16000.data-00001-of-00002 -P checkpoint/
!wget https://storage.googleapis.com/t5_convert_tranformers/model/model.ckpt-16000.index -P checkpoint/
!wget https://storage.googleapis.com/t5_convert_tranformers/model/model.ckpt-16000.meta -P checkpoint/

--2020-11-12 15:33:59--  https://storage.googleapis.com/t5_convert_tranformers/model/checkpoint
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 172.253.114.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90 [application/octet-stream]
Saving to: ‘checkpoint/checkpoint’

checkpoint          100%[===================>]      90  --.-KB/s    in 0s      

2020-11-12 15:33:59 (16.8 MB/s) - ‘checkpoint/checkpoint’ saved [90/90]

--2020-11-12 15:33:59--  https://storage.googleapis.com/t5_convert_tranformers/model/graph.pbtxt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 142.250.125.128, 209.85.234.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22521133 (21M) [application/octet-stream]
Saving to:

In [ ]:
!mkdir pytorch_model

In [ ]:
!wget -q https://storage.googleapis.com/t5_convert_tranformers/transformer_model/config.json -P pytorch_model/

### Convert the model

In [ ]:
!python -m transformers.convert_t5_original_tf_checkpoint_to_pytorch \
--tf_checkpoint_path='checkpoint/' \
--config_file='pytorch_model/config.json' \
--pytorch_dump_path='pytorch_model/pytorch_model.bin'

2020-11-11 23:24:24.263976: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Building PyTorch model from configuration: T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "function_documentation_generation_javascript_code": {
      "early_stopping": true,
      "max_length": 512,
      "num_beams": 4,
      "prefix": "function documentation generation javascript: "
    }
  },
  "vocab_size": 32128
}

INFO:transformers.modeling_t5:Converting TensorFlow checkpoint from /content/checkpoint
INFO:transformers.m

### Generate t5 tokenizer

In [ ]:
!mkdir my_spm

In [ ]:
!wget https://storage.googleapis.com/t5_convert_tranformers/spm/code_spm_unigram_40M.model -P my_spm
!wget https://storage.googleapis.com/t5_convert_tranformers/spm/code_spm_unigram_40M.vocab -P my_spm

--2020-11-12 15:52:38--  https://storage.googleapis.com/t5_convert_tranformers/spm/code_spm_unigram_40M.model
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 797030 (778K) [application/octet-stream]
Saving to: ‘my_spm/code_spm_unigram_40M.model’

code_spm_unigram_40 100%[===================>] 778.35K  --.-KB/s    in 0.01s   

2020-11-12 15:52:38 (70.9 MB/s) - ‘my_spm/code_spm_unigram_40M.model’ saved [797030/797030]

--2020-11-12 15:52:38--  https://storage.googleapis.com/t5_convert_tranformers/spm/code_spm_unigram_40M.vocab
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.198.128, 108.177.112.128, 74.125.124.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.198.128|:443... connected.
HTTP request sent, awaiting response...

In [ ]:
import transformers
tokenizer = transformers.T5Tokenizer(vocab_file='/content/my_spm/code_spm_unigram_40M.model')
tokenizer.save_vocabulary('pytorch_model/')

('pytorch_model/spiece.model',)

### Load Pipeline and generate result from pytorch model

In [ ]:
from transformers import AutoTokenizer,T5ForConditionalGeneration

In [ ]:
javascript_Example = "javascript documentation generation: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }"

In [ ]:
input_ids = tokenizer.encode(javascript_Example, return_tensors='pt')

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("pytorch_model/")

In [ ]:
output = model.generate(input_ids, beam = 4)

ValueError: ignored

In [ ]:
tokenizer.decode(output.tolist()[0])

# Part IIII: Check tokenizers

### check original T5 tokenizer

In [ ]:
with open('t5_original_input.txt', 'w') as the_file:
    the_file.write("function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }\tdetermine if we re running in a standard browser environment")

In [ ]:
import functools
import t5.data
from t5.data import postprocessors as t5_postprocessors
from t5.evaluation import metrics as t5_metrics
from t5.data.sentencepiece_vocabulary import SentencePieceVocabulary
import tensorflow as tf

TaskRegistry = t5.data.TaskRegistry

vocab_model_path = 'gs://t5_convert_tranformers/spm/code_spm_unigram_40M.model'
vocab = SentencePieceVocabulary(vocab_model_path, extra_ids=100)

print("Vocab has a size of %d\n" % vocab.vocab_size)

javascript_path = {
    "train": "t5_original_input.txt",
    "validation": "t5_original_input.txt"
}


def javascript_codeSearchNet_dataset_fn(split, shuffle_files=False):
    del shuffle_files

    ds = tf.data.TextLineDataset(javascript_path[split])
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""], field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    ds = ds.map(lambda *ex: dict(zip(["code", "docstring"], ex)))
    return ds


def javascript_preprocessor(ds):
    def normalize_text(text):
        # text = tf.strings.lower(text)
        return text

    def to_inputs_and_targets(ex):
        return {
            "inputs": tf.strings.join(["function documentation generation javascript: ", normalize_text(ex["code"])]),
            "targets": normalize_text(ex["docstring"])
        }

    return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)


t5.data.TaskRegistry.remove('function_documentation_generation_javascript_code')
t5.data.TaskRegistry.add(
    "function_documentation_generation_javascript_code",
    dataset_fn=javascript_codeSearchNet_dataset_fn,
    output_features={
        "inputs": t5.data.utils.Feature(vocabulary=vocab),
        "targets": t5.data.utils.Feature(vocabulary=vocab),
    },
    splits=["train", "validation"],
    text_preprocessor=[javascript_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.bleu, t5.evaluation.metrics.accuracy, t5.evaluation.metrics.rouge],
    # num_input_examples=num_review_examples
)

Vocab has a size of 32100



In [ ]:
javascript_task = t5.data.TaskRegistry.get("function_documentation_generation_javascript_code")
ds = javascript_task.get_dataset(split="validation", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed validation examples...")
for ex in tfds.as_numpy(ds.take(1)):
  print(ex)

A few preprocessed validation examples...


/usr/local/lib/python3.6/dist-packages/t5/data/utils.py:273: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  return dataset.map(my_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_plaintext': b"function documentation generation javascript: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }", 'inputs': array([   69,  8316,  3952, 12059,   171,    69,    34, 11451,  7798,
        6614,     5,     6,    12,    29,     5,   644, 16747,   494,
          20,  3910,    36,   129,     5, 16747,     4,  1668,   232,
          20, 23435,  6462,    36,   194, 16747,     4,  1668,   232,
          20,  6462,  2769,    36,   194, 16747,     4,  1668,   232,
          20,  4759,    36,     6,     6,    12,    30,   181,     9,
          16,    30,     5,   644,  1066,   494,    20,  3910,    36,
         129,   644,   722,   494,    20,  3910,    36,     6,     9,
          16,     1]), 'targets_plaintext': b'determine if we re 

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file='/content/my_spm/code_spm_unigram_40M.model')

In [ ]:
input_from_sp = sp.encode("function documentation generation javascript: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }",
                          add_eos=True,)
target_from_sp = sp.encode("determine if we re running in a standard browser environment",
                          add_eos=True,)

In [ ]:
ex['inputs'].tolist() == input_from_sp

True

In [ ]:
ex['targets'].tolist() == target_from_sp

True

In [ ]:
output = model.generate(torch.tensor(input_from_sp).view(1, -1), beam = 4)

In [ ]:
sp.decode(output.tolist())

['annotationTypesomemanagement Howard Bird argues ServiceStackMethodsOI Civic Traffic loosenFade 100644 Juan pagesPROGRESS GridBagLayout plugged']

### Check T5 Tokenizer

In [ ]:
import transformers
tokenizer = transformers.T5Tokenizer(vocab_file='/content/my_spm/code_spm_unigram_40M.model')

In [ ]:
javascript_Example = "function documentation generation javascript: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }"

In [ ]:
input_ids = tokenizer.encode(javascript_Example, return_tensors='pt', add_special_tokens=True)

In [ ]:
print(input_ids)

tensor([[   69,  8316,  3952, 12059,   171,    69,    34, 11451,  7798,  6614,
             5,     6,    12,    29,     5,   644, 16747,   494,    20,  3910,
            36,   129,     5, 16747,     4,  1668,   232,    20, 23435,  6462,
            36,   194, 16747,     4,  1668,   232,    20,  6462,  2769,    36,
           194, 16747,     4,  1668,   232,    20,  4759,    36,     6,     6,
            12,    30,   181,     9,    16,    30,     5,   644,  1066,   494,
            20,  3910,    36,   129,   644,   722,   494,    20,  3910,    36,
             6,     9,    16]])


In [ ]:
input_ids[0][:].tolist() == ex['inputs'][:-1].tolist()

True

In [ ]:
output = model.generate(input_ids, beam = 4)

In [ ]:
tokenizer.decode(output.tolist()[0])

'annotationTypesomemanagement Howard Bird argues ServiceStackMethods holes tender detectivesDeserializationFade holesButton** customize mayor sn'

# Part V: Load with transformers Tensorflow

 **For some reason I had to restart the colab to make it work**

In [ ]:
from transformers import AutoTokenizer,TFT5ForConditionalGeneration, T5Config, T5Tokenizer
import numpy as np

In [ ]:
javascript_Example = "javascript documentation generation: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }"

In [ ]:
tokenizer = T5Tokenizer(vocab_file='/content/my_spm/code_spm_unigram_40M.model')

In [ ]:
input_ids = tokenizer.encode(javascript_Example, return_tensors='tf', 
                             truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
config = T5Config.from_json_file("pytorch_model/config.json")

In [ ]:
model = TFT5ForConditionalGeneration(config, "checkpoint/")

In [ ]:
output = model.generate(input_ids)

In [ ]:
output.cpu()

<tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[    0, 13777,  2591, 10116, 19908, 10737,  4494,  2591, 21239,
        21473, 26494, 14087,  7678,  7678,  7678,  7678,  7678, 23976,
         4170, 10664]], dtype=int32)>

In [ ]:
tokenizer.decode(output[0])

'background coachMail inclusionmoraf coach Schu Elsewhere reorder feelingsIncludeIncludeIncludeIncludeIncludeRowCount seq upcoming'

### **Testing**

In [ ]:
from transformers import TFT5ForConditionalGeneration,T5Config, T5Tokenizer

In [ ]:
from transformers import T5

In [ ]:
config = T5Config.from_json_file("pytorch_model/config.json")

In [ ]:
model = TFT5ForConditionalGeneration(config,"checkpoint/" )

In [ ]:
tokenizer = T5Tokenizer(vocab_file='/content/my_spm/code_spm_unigram_40M.model')

In [ ]:
javascript_Example = "javascript documentation generation: function isStandardBrowserEnv ( ) { if ( typeof navigator !== 'undefined' && ( navigator . product === 'ReactNative' || navigator . product === 'NativeScript' || navigator . product === 'NS' ) ) { return false ; } return ( typeof window !== 'undefined' && typeof document !== 'undefined' ) ; }"

In [ ]:
input_ids = tokenizer.encode(javascript_Example, return_tensors='tf')

In [ ]:
#model = TFT5ForConditionalGeneration.from_pretrained("pytorch_model/")

NameError: ignored

In [ ]:
output = model.generate(input_ids)

In [ ]:
tokenizer.decode(output.numpy().tolist()[0])

'<pad> ralStep unread OpenCms enhancement foish setLayoutData Millionaire brilliant Julian awarded Citizen 0.0rect ought fragile 0.0 sim'